<a href="https://colab.research.google.com/github/chikilivighneshshastry/colab_files/blob/main/jobdata_api_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from datetime import datetime, timedelta

published_since = datetime.now() - timedelta(day=1)
print(published_since)
cookies = {
    'csrftoken': '5LSZIfHHmDnSIaZKDsN0hGYXaWQhOJGV',
}

headers = {
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9',
    'priority': 'u=1, i',
    'referer': 'https://jobdataapi.com/api/schema/swagger-ui/',
    'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    # 'cookie': 'csrftoken=5LSZIfHHmDnSIaZKDsN0hGYXaWQhOJGV',
}
parameters = {
    'country_code' : 'IN',
    # 'page' : '',
    # 'page_size' : 50,
    'published_since' : published_since
}

response = requests.get('https://jobdataapi.com/api/jobs/', cookies=cookies, headers=headers,params=parameters)

print(response)

<Response [200]>


In [ ]:
import json
# print(json.dumps(response.json()))
data = response.json()
json.dumps(response.json())

In [ ]:
results = data['results']
print(results)
len(results)

In [ ]:
import json
job_post = results[0]

json.dumps(job_post['cities'])

In [ ]:
import pandas as pd

df = pd.DataFrame(results)

df


In [ ]:
# @title experience_level vs salary_min

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['salary_min'].value_counts()
    for x_label, grp in df.groupby('experience_level')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('experience_level')
_ = plt.ylabel('salary_min')

In [ ]:
latest_published = df['published'].max()
earliest_published = df['published'].min()

print(f"Latest published date: {latest_published}")
print(f"Earliest published date: {earliest_published}")

# 3. Calculate the difference directly
# latest_published and earliest_published are already datetime objects
time_difference = latest_published - earliest_published

In [ ]:
df_city_fil = df[df['has_remote'].apply(lambda x: x)]
print(df_city_fil)

In [ ]:
df.head(2)

,id,ext_id,company,title,location,types,cities,states,countries,regions,has_remote,published,description,experience_level,application_url,language,salary_min,salary_max,salary_currency
0,22145426,QLmbEm_7Nszm,"{'id': 40577, 'name': 'Ditto Insurance', 'logo...",Associate - Business Operations,"Karnataka, Bengaluru, India","[{'id': 1, 'name': 'Full Time'}]","[{'geonameid': 1277333, 'asciiname': 'Bengalur...","[{'geonameid': 1267701, 'code': '19', 'asciina...","[{'id': 105, 'code': 'IN', 'name': 'India', 'r...","[{'id': 2, 'name': 'Asia/Pacific'}]",False,2025-06-16T14:20:56Z,"<p dir=""ltr""><span>Where Your Passion Meets Pu...",MI,https://joinditto.freshteam.com/jobs/QLmbEm_7N...,en,None,None,None
1,22144936,1sPlxCg44WeC,"{'id': 40592, 'name': 'Affinity Global Inc.', ...","Associate Team Lead, Finance","Mumbai, India","[{'id': 1, 'name': 'Full Time'}]","[{'geonameid': 1275339, 'asciiname': 'Mumbai',...","[{'geonameid': 1264418, 'code': '16', 'asciina...","[{'id': 105, 'code': 'IN', 'name': 'India', 'r...","[{'id': 2, 'name': 'Asia/Pacific'}]",False,2025-06-16T14:09:07Z,<strong> </strong><strong>About Affinity </str...,MI,https://affinity-talent.freshteam.com/jobs/1sP...,en,None,None,None


In [ ]:
company_name = df['company'][0]
print(company_name)

{'id': 40577, 'name': 'Ditto Insurance', 'logo': 'https://jobdataapi.com/media/company/logo/25/04/logo_img_nAOjTWk.png', 'website_url': 'https://joinditto.in/', 'linkedin_url': 'https://www.linkedin.com/company/ditto-insurance/', 'twitter_handle': 'joinditto', 'github_url': None, 'is_agency': False}


**Reasoning**:
Import the sqlite3 library, define the database file name, establish a connection, and create a cursor.



In [ ]:
import sqlite3

db = './jobs_data.db'
conn = sqlite3.connect(db)
cursor = conn.cursor()

## Create table if not exists

### Subtask:
Define and create the table in the database with appropriate columns if it doesn't already exist.


**Reasoning**:
Define and create the table in the database with appropriate columns if it doesn't already exist.



In [ ]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS raw_jobdataapi_data (
    id INTEGER PRIMARY KEY,
    ext_id TEXT,
    company TEXT,
    title TEXT,
    location TEXT,
    types TEXT,
    cities TEXT,
    states TEXT,
    countries TEXT,
    regions TEXT,
    has_remote BOOLEAN,
    published TEXT,
    description TEXT,
    experience_level TEXT,
    application_url TEXT,
    language TEXT,
    salary_min INTEGER,
    salary_max INTEGER,
    salary_currency TEXT
)
"""
cursor.execute(create_table_sql)
conn.commit()

**Reasoning**:
Iterate through the DataFrame, check for existing records by id, and insert new records into the SQLite database, converting complex types to JSON strings.



In [ ]:
import json
import pandas as pd
import sqlite3
from datetime import datetime, timedelta


published_since = datetime.now() - timedelta(day=1)
db = './jobs_data.db'
conn = sqlite3.connect(db)

df = df[df['published']<20min] # correct this code

# Get existing IDs from the database
existing_ids = pd.read_sql(f"SELECT id FROM raw_jobdataapi_data where published > {published_since}", conn)['id'].tolist() # get last 20 min id's

# Filter out rows that already exist in the database
df_to_insert = df[~df['id'].isin(existing_ids)].copy()

# Convert complex data types to JSON strings in the DataFrame
for col in ['company', 'types', 'cities', 'states', 'countries', 'regions']:
    df_to_insert[col] = df_to_insert[col].apply(lambda x: json.dumps(x) if isinstance(x, (dict, list)) else None)

# Insert the filtered DataFrame into the SQLite database
df_to_insert.to_sql('raw_jobdataapi_data', conn, if_exists='append', index=False)

conn.commit()

**Reasoning**:
Close the connection to the SQLite database.



In [ ]:
conn.close()

In [ ]:
# read data from this raw_table (last 20 min data)
# read main_jobs_data table (last 20 min data)
# compare both and remove and remove commen
# for unique run data_manupulation worker
# get city and (location details) , work_type,clean published (date) and check if date is behyod the current data, from json
# description -->html to text , company details from json to compnay table mapping
# send data to ai to tag a category,get skills,yeas_of_exp (min and max),musthave qulification,prefred qulifications,,

#save this data in main jobs table and also inject data in main jobs dependcy tables like category and jobpost matching

In [ ]:
# when user request data
# get all jobpost from main jobs table after filtering category (selected by user)
# check if this jobposts already in user_score table if not run scoring worker and append data to table
# send data to front end
# delete expried jobpost fromt his table when delete_expried_worker run

In [ ]:
import json
import pandas as pd
import sqlite3

db = './jobs_data.db'
conn = sqlite3.connect(db)
cursor = conn.cursor()
create_table_sql = """
CREATE TABLE IF NOT EXISTS job_posts (
    id TEXT PRIMARY KEY,
    jobtitle TEXT,
    jobdataapi_id TEXT,
    title TEXT,
    location TEXT,
    jobdescription TEXT,
    company_id TEXT,
    published_date TEXT, -- Storing ISO 8601 strings, e.g., 'YYYY-MM-DD HH:MM:SS'
    published_time_ux INTEGER, -- Unix timestamp (seconds since epoch)
    experience_level TEXT,
    salary_min INTEGER,
    salary_max INTEGER,
    salary_currency TEXT,
    job_language TEXT,
    application_url TEXT,
    -- Define Foreign Key constraint (optional if company_details table isn't created yet)
    FOREIGN KEY (company_id) REFERENCES company_details(id)
        ON DELETE NO ACTION ON UPDATE NO ACTION
);
"""
# Execute the SQL command
cursor.execute(create_table_sql)
conn.commit()
print(f"Table 'job_posts' created successfully in {db}")

In [ ]:
main_table_ids = pd.read_sql(f"SELECT jobdataapi_id FROM job_posts where published > {published_since}", conn)['jobdataapi_id'].tolist()
raw_table_ids = pd.read_sql(f"SELECT id FROM raw_jobdataapi_data where published > {published_since}", conn)['id'].tolist()
# Filter out rows that already exist in the database
df_to_insert = df[~df['id'].isin(existing_ids)].copy()